## **Multiplicação de Matrizes:** Implementação _sequencial_, _Multithreading_ e _Multiprocessos_.

***

Este projeto visa observar as diferenças entre a implementação de algoritmos utilizando as abordagens sequencial, Multithreading e Multiprocessos, realizando testes de implementação e comparativos em termos de desempenho utilizando a linguagem **Python**, para a disciplina **Sistemas Operacionais** do curso de Bacharelado em Tecnologia da Informação, oferecido pela Universidade Federal do Rio Grande do Norte.

**Desenvolvido por:** José Manoel Freitas da Silva

***

### **1. Introdução**

A multiplicação de matrizes é uma operação fundamental em álgebra linear que desempenha um papel crucial em diversos campos da ciência da computação, engenharia e ciências exatas. Uma das principais vantagens da utilização desse método é a sua natureza altamente paralelizável, uma vez que multiplicação de matrizes é bastante suscetível à distribuição de tarefas, seja por meio de threads, em um ambiente multithread, ou por processos. Além disso, a multiplicação de matrizes pode ser escalada para grandes conjuntos de dados e dimensões, o que a torna relevante para problemas de alta complexidade encontrados em pesquisa científica e aplicações industriais.

Desse modo, iremos utilizar de uma de suas propriedades, chamada **independência de dados**, que nos permite calcular cada elemento da matriz resultante de forma independente dos demais dados da matriz, para implementar diversos conceitos da multiprogramação e analisar questões fundamentais de desempenho e otimização algorítmica, ressaltando a sua importância prática e sua capacidade de aproveitar ao máximo os recursos computacionais modernos. 

### **2. Metodologia**

Para esse projeto será feito 3 implementações, sendo elas:

* Uma implementação **Sequencial**, sem a utilização de multiprocessos ou multithreading;
* Uma implementação utilizando **Threads**;
* Uma implementação utilizando **Processos**.

Cada uma das implementações receberá duas matrizes **M1** e **M2** de tamanho **C**x**L** variados, com valores e dimensões randomizadas por meio de um algorítimo auxiliar implementado em **Python**. Para as implementações em multiprocessamento será utilizado ainda uma segunda variável **P**, que determina a quantidade de elementos que serão processados em cada thread ou processo criado, de modo a evitar problemas oriundos do excesso de processos abertos, frequentemente vistos em matrizes consideravelmente grandes.

Para quantificar o desempenho de cada implementação será medido o tempo de execução em cada uma das metodologias, realizando a plotagem de gráficos de desempenho com o intuito de visualizar o **tempo médio geral** e o **tempo médio em função de P**.

### **3. Considerações iniciais**

Nesse projeto, devido ao seu caráter analítico, será utilizado a linguagem de programação **Python**, aliada ao **Jupyter Notebook**, onde iremos observar o comportamento de diferentes formas de implementação de um mesmo algoritmo com o intuito de analisar as vantagens e desvantagens de **aplicações sequenciais** e **paralelas**.

A escolha dessas duas ferramentas se deve ao fato de permitir produzir gráficos e relatórios com facilidade quando utilizadas em conjunto com algumas bibliotecas, como a _multiprocessing_, _threading_,_matplotlib_ e a _numpy_, que terá um papel fundamental na criação de objetos e elementos utilizados durante os testes.

### **4. Multiplicação de matrizes**

#### **4.1. Operações entre matrizes**

Uma matriz pode ser definida como uma estrutura matemática que organiza elementos em **linhas** e **colunas**, formando uma grade bidimensional. Nela cada elemento é identificado por sua posição única na matriz através do índice de linhas e colunas. Por exemplo, na matriz $A$, de dimensões 3x3, o elemento $A[i][j]$ é identificado pela linha $i$ e coluna $j$.

\begin{equation*}

A = \begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9

\end{bmatrix}

\end{equation*}

Assim, o elemento $A_{1,2}$ da matriz $A$ é igual a **2**.

Essa estrutura também está suscetível a algumas operações, como a multiplicação entre matrizes, que será utilizada como objeto de estudo.

A **multiplicação de matrizes** é uma operação que corresponde ao  produto de duas matrizes $A$ e $B$, para produzir uma terceira matriz $C$. Para calcular o elemento $C_{i,j}$ da matriz resultante $C$, devemos multiplicar os elementos da linha $i$ da matriz $A$ pelos elementos da coluna $j$ da matriz $B$, somando os produtos. Matematicamente, isso pode ser expresso como:

\begin{equation*}
C_{i,j} = A_{i,1} \cdot B_{1,j} + A_{i,2} \cdot B_{2,j} + \ldots + A_{i,n} \cdot B_{n,j}
\end{equation*}

Esse processo é repetido para todos os elementos da matriz resultante $C$, e o resultado é uma nova matriz que é o produto das matrizes $A$ e $B$, com a quantidade colunas de $A$ e linhas de $B$, assim como veremos nos algoritmos implementados a seguir.

### **5. Algoritmos e implementações** 

#### **5.1. Importando recursos**

Para a implementação dos algoritmos a seguir utilizaremos as seguintes bibliotecas:

* **numpy:** Biblioteca em Python que fornece suporte para arrays multidimensionais e funções matemáticas de alto desempenho para manipulação de dados numéricos;
* **numba:** Biblioteca em Python que oferece uma forma de acelerar código Python usando compilação just-in-time (JIT);
* **time:** Biblioteca em Python utilizada para lidar com operações relacionadas ao tempo, como o tempo de execução de algoritmos;

A biblioteca **Numba** terá um importante papel na implementação multithread com python, sendo revisitada posteriormente

In [1]:
import numba as nb
import numpy as np
import time

#### **5.2. Criando matrizes**

Nesse estudo serão utilizadas matrizes de inteiros construídas de forma randômica através da função **_get_random_matrix_**, que utiliza a função **_random_**, da biblioteca **NumPy**. Para fins de estudos todos os elementos irão possuir um valor fixado entre 1 e 10. Além das matrizes randômicas, também iremos utilizar matrizes inicializadas com zeros por meio da função **_initialize_matrix_**, necessária para garantir que todos os elementos da matriz resultante tenham um valor inicial conhecido e consistente.

Vale ressaltar que para ocorrer a multiplicação entre matrizes a quantidade de linhas de $A$ e colunas de $B$ deve possuir o mesmo valor, seguindo a propriedade a seguir:

\begin{equation*}
[A_{m \times n}] \cdot [B_{n \times p}] = [C_{m \times p}]
\end{equation*}

Desse modo, para satisfazer essa condição, iremos utilizar a variável **_cond_** para garantir que ambas as matrizes criadas obedeçam essa propriedade.

In [2]:
def get_random_matrix(rows, cols):
    # Cria matrizes aleatórias com valores 1 e 10
    return np.random.randint(1, 11, size = (rows, cols))

In [3]:
# Limite para o tamanho das matrizes
limit = 1000

# Condição para existencia do produto
cond = np.random.randint(2, limit)

# Cria matrizes de tamanhos aleatórios
matrix1 = get_random_matrix(np.random.randint(2, limit), cond)
matrix2 = get_random_matrix(cond, np.random.randint(2, limit))

#### **5.3. Matriz resultante**

Com a finalidade de facilitar a visualização de cada resultado em diferentes implementações, será utilizado um objeto **_Resultante_** para armazenar os dados obtidos em cada execução, composto por:

* **matrix:** matriz resultante;
* **process_time:** tempo de execução;
* **P:** quantidade de elementos que serão processados em cada thread ou processo criado.

Onde **_P_** receberá valor **-1** caso seja uma execução sequencial.

In [4]:
class Resultante:
    def __init__(self, matrix, process_time, P = -1):
        self.matrix = matrix
        self.process_time = process_time
        self.P = P

### **6. Implementação Sequencial**

A implementação sequencial a seguir é responsável por multiplicar duas matrizes, **_matrix1_** e **_matrix2_**, de acordo com o algoritmo tradicional de multiplicação utilizando três loops aninhados. O primeiro loop (**_i_**) itera pelas linhas da matriz resultante, o segundo loop (**_j_**) itera pelas colunas da matriz resultante e o terceiro loop (**_k_**) itera pelas colunas da primeira matriz e pelas linhas da segunda matriz. Durante cada iteração, os elementos correspondentes das matrizes de entrada são multiplicados e acumulados no elemento correspondente da matriz resultante.

Após a conclusão da multiplicação, o tempo total de execução é calculado e é retornado um objeto **_Resultante_** que contém a matriz resultante e o tempo total de execução.

In [5]:
def sequential_multiplication(matrix1, matrix2):
    # Inicializa a matriz resultante com 0
    matrix_result = np.zeros((matrix1.shape[0], matrix2.shape[1]))
    
    # Inicializa a contagem de tempo
    start = time.time()

    # Realiza a multiplicação de matrizes
    for i in range(matrix1.shape[0]):
        for j in range(matrix2.shape[1]):
            for k in range(matrix1.shape[1]):
                matrix_result[i][j] += matrix1[i][k] * matrix2[k][j]
                
    # Finaliza a contagem de tempo
    end = time.time()

    # Tempo total de execução
    total_time = end - start

    # Cria o objeto resultante
    result = Resultante(matrix_result, total_time)

    return result

### **7. Implementação com Threads**

#### **7.1. Multithread e o Global Interpreter Lock (GIL)**

O Python possui uma maneira diferente de lidar com variáveis criadas em ambiente, considerando-as apenas como uma marcação para referência do local em memória. Assim, ao atribuir novas variáveis, o que realmente ocorre é apenas uma passagem de endereços em memória, sem uma tipagem real.

Essa abordagem permitiu ao Python adquirir flexibilidade com a passagem e atribuição de variáveis, no entanto, também acarretou num sério problema de memory safety, observável quando dois ou mais processos tentam acessar a mesma variável, causando uma condição de corrida.

Para solucionar o problema anterior sem perder a flexibilidade, o Python optou por implementar o Python Global Interpreter Lock (GIL), que funciona como um bloqueio de processo, utilizado para garantir a integridade da aplicação que está sendo executada ao travar parte da aplicação para garantir que apenas uma chamada tenha acesso à memória por vez. Desse modo, ao bloquear parte da execução do programa também temos uma perda significativa de desempenho ao executar processos em multithread, uma vez que teremos uma série de interrupções, resultando num desempenho semelhante a uma aplicação single-thread.

#### **7.2. Biblioteca Numba**

Para contornar o uso do GIL e permitir o processamento em multithread é comum recorrer técnicas e bibliotecas que ofereçam um compilador alternativo, como o Numba. O Numba é uma biblioteca que traz um compilador just-in-time (JIT) para o Python, projetado para melhorar o desempenho de aplicações compilando o código-fonte diretamente para código de máquina em tempo de execução, usando a infraestrutura do compilador LLVM (Low Level Virtual Machine). 

Essa prática permite alcançar desempenho semelhante ao C ou C++ sem a necessidade de alterar o código inicial, além de também suportar aceleração por GPU, como a utiliza na API CUDA, da NVidea. 

Para utilizar a biblioteca Numba devemos adicionar um decorator _@jit_, que designa que a função deve ser compilada através do JIT, além de conter outros atributos, sendo eles:

* **_noPython:_** Sinaliza que a função deve ser executada sem utilizar o interpretador padrão do Python;
* **_nogil:_** Indica que o GIL deve ser liberado durante a execução, alcançando o uso efetivo das múltiplas threads do sistema;
* **_parallel:_** Sinaliza que a função deve utilizar execução paralela;
* **_cache:_** Explicita que o compilador deve armazenara função em memória cache, evitando que o código seja recompilado em execuções posteriores.

O Numba por padrão está configurado para utilizar o máximo de threads disponíveis no sistema, descartando a necessidade de configurá-las. Ao final o decorator deve possuir uma sintaxe semelhante a apresentada a seguir:

~~~python
@nb.jit(nopython=True, parallel=True, nogil=True, cache=True)
def multiply_submatrix(submatrix1, submatrix2):
~~~

#### **7.3. Implementação em Python**

A implementação do algoritmo de multiplicação de matrizes com threads segue uma abordagem que utiliza a biblioteca Numba para contornar as limitações do Python, incluindo o Global Interpreter Lock (GIL), que impede a execução simultânea de código Python por várias threads.

O código foi dividido em duas funções, sendo **_multiply_submatrix_** a função é responsável por realizar a multiplicação de submatrizes de forma paralela com a paralelização do Numba e a **_parallel_multiplication_**, que realiza a multiplicação de matrizes em paralelo dividindo o cálculo em blocos de tamanho PxP de submatrizes, que ao final são combinadas para formar a matriz resultante final.

In [6]:
# Função para multiplicação de submatrizes em paralelo com Numba
@nb.jit(nopython=True, parallel=True, nogil=True, cache=True)
def multiply_submatrix(submatrix1, submatrix2):
    # Submatriz criada para o calculo
    subresult = np.zeros((submatrix1.shape[0], submatrix2.shape[1]))

    for i in nb.prange(submatrix1.shape[0]):
        for j in nb.prange(submatrix2.shape[1]):
            for k in range(submatrix1.shape[1]):
                subresult[i][j] += submatrix1[i][k] * submatrix2[k][j]
    
    return subresult

In [7]:
# Função de multiplicação de matrizes paralelizada
def parallel_multiplication(matrix1, matrix2, P):
    # Inicializa a matriz resultante
    matrix_result = np.zeros((matrix1.shape[0], matrix2.shape[1]))
    
    # Inicializa a contagem de tempo
    start = time.time()

    # Realiza a multiplicação de matrizes em paralelo com blocos de tamanho PxP
    for i in nb.prange(0, matrix1.shape[0], P):
        for j in nb.prange(0, matrix2.shape[1], P):

            submatrix1 = matrix1[i:i+P, :]
            submatrix2 = matrix2[:, j:j+P]

            subresult = multiply_submatrix(submatrix1, submatrix2)

            matrix_result[i:i+P, j:j+P] = subresult

    end = time.time()

    # Tempo total de execução
    total_time = end - start

    # Cria o objeto resultante
    result = Resultante(matrix_result, total_time)

    return result


In [8]:
result = parallel_multiplication(matrix1, matrix2, 50)
print(result.matrix)
print(result.process_time)

print("\n")

result_1 = sequential_multiplication(matrix1, matrix2)
print(result_1.matrix)
print(result_1.process_time)

[[ 932.  996.  924. ...  971. 1007.  962.]
 [ 997. 1040.  892. ... 1097. 1055. 1121.]
 [ 859.  933.  658. ...  901.  786.  839.]
 ...
 [1031. 1097. 1024. ... 1101. 1124. 1112.]
 [ 946.  846.  712. ...  869. 1014.  970.]
 [ 862.  925.  859. ...  974. 1009.  906.]]
2.2042086124420166




[[ 932.  996.  924. ...  971. 1007.  962.]
 [ 997. 1040.  892. ... 1097. 1055. 1121.]
 [ 859.  933.  658. ...  901.  786.  839.]
 ...
 [1031. 1097. 1024. ... 1101. 1124. 1112.]
 [ 946.  846.  712. ...  869. 1014.  970.]
 [ 862.  925.  859. ...  974. 1009.  906.]]
27.01604151725769


### **8. Implementação com processos**